In [1]:
# python imports
import re
import json
from os.path import join
import cPickle
import numpy as np
import math
import itertools

# third party imports\
import pandas
import escher
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import (ribosome, tRNA_charging, transcription, translocation, 
                     chaperones, DNA_replication, translation)
from ecolime.characterization.biomass_allocation import get_membrane_composition, get_membrane_protein

from cobrame import *
from cobrame.util import building, dogma
from cobrame.util.mass import compute_RNA_mass
from cobrame.solve.algorithms import binary_search, fva, solve_at_growth_rate
from cobrame.solve.symbolic import compile_expressions

from cloudpickle import load
import cPickle

from cobrame import ComplexData, TranscribedGene, ModificationData
from cobrame.util.building import add_modification_data

import cobra
import copy
from copy import deepcopy
from tqdm import tqdm
import pandas as pd

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def find_trna_enzymes():
    """
    finds all enzymes used in tRNA modifications
    """
    results_list = []
    for tRNA_modification in tRNA_charging.trna_modification:
        tRNA_enzyme_list = tRNA_charging.trna_modification[tRNA_modification]['machines'] 
        
        
        for tRNA_enzyme in tRNA_enzyme_list:
            if tRNA_enzyme not in results_list and tRNA_enzyme != None :
                results_list.append(tRNA_enzyme)
    return results_list
#find_trna_enzymes()

In [3]:
#needs to be changed to reflect individual cases for each sweep
#needs to be modified for tRNA_enzyme_KO
def create_tRNA_modification_data(me, type_tRNA_enzyme_sweep = False, tRNA_enzyme_KO = False):
    for mod, components in get_tRNA_modification_procedures(type_tRNA_enzyme_sweep, tRNA_enzyme_KO).items():
        tRNA_mod = ModificationData(mod, me)

        tRNA_mod.enzyme = components['machines']
        tRNA_mod.stoichiometry = components['metabolites']
        tRNA_mod.keff = 65. # iOL uses 65 for all tRNA mods
        if 'carriers' in components.keys():
            for carrier, stoich in components['carriers'].items():
                if stoich < 0:
                    tRNA_mod.enzyme += [carrier]
                tRNA_mod.stoichiometry[carrier] = stoich
                #adds carriers as metabolites
                #adds carriers as enzymes if stoich < 0 

    # tRNA_modifications = {tRNA_id: {modifications: count}}
    tRNA_modifications = get_tRNA_modification_targets(type_tRNA_enzyme_sweep, tRNA_enzyme_KO)
    for tRNA in tRNA_modifications:
        for data in me.tRNA_data.query(tRNA):
            data.modifications = tRNA_modifications[tRNA]
    return me
            

#create_tRNA_modification_data()        
        #assigns modifications to each tRNA_id (bnumber)

In [ ]:
def create_proto_for_enzyme_sweep(type_tRNA_enzyme_sweep = False, rRNA_modifications =  False):
    from tqdm import tqdm
    count = 0
    for tRNA_enzyme_KO in tqdm(find_trna_enzymes()):
        count = count + 1
        print count,  tRNA_enzyme_KO
        with open('../prototype_notebooks/RNA_sweep_control_prototypes/proto_65_NO_tRNA_mods_NO_rRNA_mods_NO_tRNA_data.pickle', 'rb') as f:
            me = cPickle.load(f)

        create_tRNA_modification_data(me, type_tRNA_enzyme_sweep, tRNA_enzyme_KO)


        # rrna_modifications added later
        ribosome.add_ribosome(me, verbose=False, rRNA_mods = rRNA_modifications) # <<<<<<<<<<<<<<----------rRNA mods go HERE
        #RNA_mods = True adds the modifications
        #me.reactions.formation_ribosome.reaction

        # These should be coupled to translation not ribosome formation per iOL1650
        Inf_modifications = ['Translation_initiation_factor_InfA', 'Translation_gtp_initiation_factor_InfB',
                        'Translation_initiation_factor_InfC']
        for sub in Inf_modifications:
            me.complex_data.ribosome.modifications[sub] = 0
            for data in me.translation_data:
                data.modifications[sub]= -1

        me.update()
        me.prune()
        
        #MODIFY HERE TO CHANGe FOR POSITIVE/NEGATIVE
        save_path = './tRNA_sweeps/tRNA_positive_enzyme_sweep/prototypes/'
        filename = 'proto_65_tRNA_pos_enzyme_sweep_' + str(tRNA_enzyme_KO) + '_rRNA_mods_' + str(rRNA_modifications) + '.pickle'
        
        with open(save_path + filename, 'w') as outfile:
            cPickle.dump(me, outfile)


In [ ]:
for rRNA_mod in [True, False]:
    create_proto_for_enzyme_sweep(type_tRNA_enzyme_sweep = 'positive' , rRNA_modifications = rRNA_mod)

  3%|▎         | 1/37 [01:56<1:09:54, 116.52s/it]

1 TmcA_mono
2

  5%|▌         | 2/37 [03:54<1:08:16, 117.04s/it]

 RlmN_mono_mod_1:4fe4s
3

  8%|▊         | 3/37 [05:54<1:06:48, 117.89s/it]

...

 TrmU_mono
12

 32%|███▏      | 12/37 [24:32<53:56, 129.45s/it]

 TsaBCDE
13

 35%|███▌      | 13/37 [27:10<55:17, 138.23s/it]

 generic_Dus
14

# trouble shooting